In [1]:
! pip install -U spacy -q

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
import json
f = open('/content/data+new_data+13entities_annotations+name_age.json')
TRAIN_DATA = json.load(f)

In [4]:
TRAIN_DATA


{'classes': ['DISEASE',
  'SYMPTOM',
  'MEDICINE',
  'QUANTITY',
  'HEIGHT',
  'WEIGHT',
  'TEMPERATURE',
  'HEART RATE',
  'SPO2',
  'BLOOD PRESSURE',
  'BODY PARTS',
  'MOBILE NUMBER',
  'PINCODE'],
 'annotations': [['A typical measurement error of plus or minus 0.5 cm (0.20 in) may completely nullify 0.5  cm of actual growth resulting in either a "negative" 0.5  cm growth (due to overestimation in the previous visit combined with underestimation in the latter), up to a 1.5 cm (0.6 in) growth.\r',
   {'entities': [[45, 51, 'HEIGHT'],
     [53, 60, 'HEIGHT'],
     [85, 92, 'HEIGHT'],
     [143, 150, 'HEIGHT'],
     [257, 263, 'HEIGHT'],
     [265, 271, 'HEIGHT']]}],
  ['He observed that Mayan Indian men averaged 157.5 centimetres (5 ft 2 in) in height.\r',
   {'entities': [[43, 60, 'HEIGHT'], [62, 71, 'HEIGHT']]}],
  ['The women averaged 142.2 centimetres (4 ft 8 in).\r',
   {'entities': [[19, 36, 'HEIGHT'], [38, 47, 'HEIGHT']]}],
  ['Dinka Ruweng males investigated by Roberts in 1953

In [5]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 391/391 [00:00<00:00, 1736.37it/s]


In [6]:
! python -m spacy init config config.cfg --lang en --pipeline ner  --optimize efficiency

2022-09-04 09:48:14.837368: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2022-09-04 09:48:21.963434: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-09-04 09:48:22,760] [INFO] Set up nlp object from config
[2022-09-04 09:48:22,772] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-09-04 09:48:22,778] [INFO] Created vocabulary
[2022-09-04 09:48:22,779] [INFO] Finished initializing nlp object
[2022-09-04 09:48:23,240] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     38.34    0.00    0.00    0.00    0.00
  1     200       2312

In [12]:
nlp_ner = spacy.load("/content/model-best") 

In [13]:
doc = nlp_ner(''' Shahid Afridi is 53 years old. Symptoms are highly variable and may include, persistent lump, weight loss and other unexplained changes in the body.A lump or mass in the breast that feels different from the surrounding tissue causes liver cancer,can be treated with Dolo (650mg) tablets ith 120 mmHg , 150cm , 5ft5in, 100kg, 150 bpm, 90%, 9589408732, 583201, 93.2 °F, 22°c ''') # input sample text

In [14]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [21]:
a = nlp_ner('''Shriya Ellur is 25 years old''')

In [22]:
spacy.displacy.render(a, style="ent", jupyter=True)